Project Proposal

Title:


Introduction:

Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
Clearly state the question you will try to answer with your project
Identify and describe the dataset that will be used to answer the question


Preliminary exploratory data analysis:

Demonstrate that the dataset can be read from the web into R 
Clean and wrangle your data into a tidy format
Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

Methods:

Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize the results

Expected outcomes and significance:

What do you expect to find?
What impact could such findings have?
What future questions could this lead to?

In [19]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [20]:
attrition_data <- read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
attrition_data

Rows: 1470 Columns: 35
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): Attrition, BusinessTravel, Department, EducationField, Gender, Job...
dbl (26): Age, DailyRate, DistanceFromHome, Education, EmployeeCount, Employ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,⋯,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,⋯,1,80,0,8,0,1,6,4,0,5
49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,⋯,4,80,1,10,3,3,10,7,1,7
37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,⋯,2,80,0,7,3,3,0,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,⋯,2,80,1,6,0,3,6,2,0,3
49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,⋯,4,80,0,17,3,2,9,6,0,8
34,No,Travel_Rarely,628,Research & Development,8,3,Medical,1,2068,⋯,1,80,0,6,3,4,4,3,1,2


In [21]:
attrition_data <- attrition_data |>
    select( Attrition, HourlyRate, YearsAtCompany)

attrition_data

Attrition,HourlyRate,YearsAtCompany
<chr>,<dbl>,<dbl>
Yes,94,6
No,61,10
Yes,92,0
⋮,⋮,⋮
No,87,6
No,63,9
No,82,4


In [22]:
attrition_data <- attrition_data |>
        mutate(Attrition = as_factor(Attrition))

In [37]:
attrition_data_scaled <- attrition_data |> 
 mutate(scaled_hourly_rate = scale(HourlyRate, center = TRUE), 
        scaled_years_at_company = scale(YearsAtCompany, center = TRUE))
       
attrition_data_scaled

Attrition,HourlyRate,YearsAtCompany,scaled_hourly_rate,scaled_years_at_company
<fct>,<dbl>,<dbl>,"<dbl[,1]>","<dbl[,1]>"
Yes,94,6,1.3826677,-0.1645571
No,61,10,-0.2405949,0.4883415
Yes,92,0,1.2842882,-1.1439051
⋮,⋮,⋮,⋮,⋮
No,87,6,1.0383393,-0.1645571
No,63,9,-0.1422153,0.3251169
No,82,4,0.7923904,-0.4910064


In [39]:
attrition_split <- initial_split(attrition_data_scaled, prop = 0.75, strata = Attrition)
attrition_train <- training(attrition_split)
attrition_test <- testing(attrition_split) 

glimpse(attrition_train)
glimpse(attrition_test)

Rows: 1,101
Columns: 5
$ Attrition               <fct> No, No, No, No, No, No, No, No, No, No, No, No…
$ HourlyRate              <dbl> 61, 56, 40, 44, 94, 84, 49, 93, 80, 96, 78, 45…
$ YearsAtCompany          <dbl> 10, 8, 2, 9, 7, 5, 9, 2, 6, 1, 25, 3, 4, 0, 9,…
$ scaled_hourly_rate      <dbl[,1]> <matrix[26 x 1]>
$ scaled_years_at_company <dbl[,1]> <matrix[26 x 1]>
Rows: 369
Columns: 5
$ Attrition               <fct> No, No, No, No, No, No, No, No, Yes, No, Yes, …
$ HourlyRate              <dbl> 79, 81, 67, 31, 51, 53, 58, 41, 61, 42, 50, 71…
$ YearsAtCompany          <dbl> 7, 1, 1, 5, 10, 12, 14, 4, 2, 5, 2, 9, 2, 6, 3…
$ scaled_hourly_rate      <dbl[,1]> <matrix[26 x 1]>
$ scaled_years_at_company <dbl[,1]> <matrix[26 x 1]>


In [40]:
attrition_proportions <- attrition_train |>
                      group_by(attrition) |>
                      summarize(n = n()) |>
                      mutate(percent = 100*n/nrow(attrition_train))

attrition_proportions

attrition_proportions_train <- attrition_train |>
                      group_by(Attrition) |>
                      summarize(n = n()) |>
                      mutate(percent = 100*n/nrow(attrition_train))

attrition_proportions_train

Attrition,n,percent
<fct>,<int>,<dbl>
Yes,177,16.07629
No,924,83.92371


Attrition,n,percent
<fct>,<int>,<dbl>
Yes,177,16.07629
No,924,83.92371
